# 1. Import of input data

In [ ]:
import nbimporter
import sys
import os
from pathlib import Path

In [ ]:
# Import libraries
try:
    current_dir = RUN_DIR
    print(f"📂 CoolingEvaporativeTower.ipynb executed by RUN.ipynb — using directory: {current_dir}")
except NameError:
    current_dir = Path().resolve()
    print(f"📁 CoolingEvaporativeTower.ipynb executed standalone — using directory: {current_dir}")

Libraries_path = (current_dir / "Libraries.ipynb").resolve()

# If not found, try in the parent levels
if not Libraries_path.exists():
    Libraries_path = (current_dir.parent / "Libraries.ipynb").resolve()

# Run the file
if Libraries_path.exists():
    print(f"📘 Importing libraries from: {Libraries_path}")
    get_ipython().run_line_magic('run', f'"{Libraries_path}"')
else:
    print("⚠️ Libraries.ipynb not found! Some functions might not be available.")

In [ ]:
# Database parameter import
energysystem = pd.read_excel(energysystem_path, sheet_name = 'Cooling - Cooling tower')

# 2. Input parameters

In [ ]:
# DASHBORD - VALUE INPUTS
Power = energysystem['Cooling capacity [kW]']
EnergyCooling = energysystem['Energy produced [kWh /year]']
COP = energysystem['COP']
ElectricityGrid = energysystem['Energy input from national grid [kWh/year]']
ElectricityPV = energysystem['Energy input from Photovoltaic system [kWh/year]']
ElectricityRICE = energysystem['Energy input from CHP-Internal Combustion Engine [kWh/year]']
ElectricityCombinedCycle = energysystem['Energy input from CHP-Combined cycle [kWh/year]']
ElectricityGasTurbine = energysystem['Energy input from CHP-Gas Turbine [kWh/year]']
ElectricityWtE = energysystem['Energy input from CHP-Waste-to-Energy [kWh/year]']
ElectricityORC = energysystem['Energy input from CHP-Organic Rankine Cycle [kWh/year]']

In [ ]:
# DASHBOARD - INPUT PARAMETERS
massRef = 3500 #kg
PowerRef = 300 # kW (Source: Ecoinvent database)
mass = massRef * (Power / PowerRef)
Lifespan = 20 # year

# 3. Life cycle inventories

## 3.1 Module A1-3

In [ ]:
CoolingTower_stainlessSteel = 0.4738 * mass
CoolingTower_PVC = 0.10822 * mass
CoolingTower_castiron = 0.3376 * mass
CoolingTower_airfun = 0.0760 * mass
CoolingTower_sheetrollingchromiumsteel = 0.4738 * mass

## 3.2 Module A4

In [ ]:
CoolingTower_Truk_16_32 = ((500 + 200)*(mass)/1000)
CoolingTower_Light_Truk = (50*(mass)/1000)

## 3.3 Module B2

In [ ]:
CoolingTower_Light_Truk_B2 = 50 * (Lifespan/2) * 0.01 * (mass/1000)

## 3.4 Module B4

In [ ]:
Lifespan = 25 #year Hampoo et al. (2021)
CoolingTowerNumberReplacements = (np.ceil(np.divide(ReqSL, Lifespan) - 1))
SFCoolingTower = (RSP / ReqSL)  # Scaling factor

## 3.5 Module B6

In [ ]:
#Parameters for energy input repartition
TotalElectricityInput = (ElectricityGrid + ElectricityPV + ElectricityRICE + ElectricityCombinedCycle +
    ElectricityGasTurbine + ElectricityWtE + ElectricityORC)

CoolingTower_EleGrid = ElectricityGrid / TotalElectricityInput
CoolingTower_ElePV = ElectricityPV / TotalElectricityInput
CoolingTower_EleRICE = ElectricityRICE / TotalElectricityInput
CoolingTower_EleCombinedCycle = ElectricityCombinedCycle / TotalElectricityInput
CoolingTower_EleGasTurbine = ElectricityGasTurbine / TotalElectricityInput
CoolingTower_EleWtE = ElectricityWtE / TotalElectricityInput
CoolingTower_EleORC = ElectricityORC / TotalElectricityInput

In [ ]:
CoolingTower_usephaseelectricity = (1 / COP) * EnergyCooling * RSP * CoolingTower_EleGrid * Grid
CoolingTower_usephaseelectricityPVExt = (1 / COP) * EnergyCooling * RSP * CoolingTower_EleGrid * PV
CoolingTower_usephaseelectricityPV = (1 / COP) * EnergyCooling * RSP * CoolingTower_ElePV

CoolingTower_usephaseelectricityRECSPV = CoolingTower_usephaseelectricity * 0.0056 * RECSPV
CoolingTower_usephaseelectricityRECSHydro = CoolingTower_usephaseelectricity * 0.0056 * RECSHydro
CoolingTower_usephaseelectricityRECSEolic = CoolingTower_usephaseelectricity * 0.0056 * RECSEolic

CoolingTower_waterdemand = 63740 * (Power / PowerRef) #m3
CoolingTower_wastewater = 0.03 * 1000 * CoolingTower_waterdemand

## 3.6 Module C2

In [ ]:
CoolingTower_Transport_to_treatment_plant = 50 * (mass/1000)

## 3.7 Module C3

In [ ]:
CoolingTower_EOL_stainlessSteel = CoolingTower_stainlessSteel
CoolingTower_EOL_PVC = CoolingTower_PVC
CoolingTower_EOL_castiron = CoolingTower_castiron
CoolingTower_EOL_airfun = CoolingTower_airfun

## 3.8 Module C4

In [ ]:
CoolingTower_EOL_landfill_stainlessSteel = CoolingTower_EOL_stainlessSteel * LandfillMetals2
CoolingTower_EOL_inc_PVC = CoolingTower_EOL_PVC * IncinerationPVC
CoolingTower_EOL_landfill_PVC = CoolingTower_EOL_PVC * LandfillPVC
CoolingTower_EOL_landfill_castiron = CoolingTower_EOL_castiron * LandfillMetals
CoolingTower_EOL_landfill_airfun = CoolingTower_EOL_airfun * LandfillMetals2

## 3.9 Module D1 (related to the export of secondary materials)

In [ ]:
CoolingTower_EOL_recycling_stainlessSteel = (RecyclingMetals2 * RecyclingEfficiencyMetals * CoolingTower_EOL_stainlessSteel) - (RecycledContentMetals * CoolingTower_EOL_stainlessSteel)
CoolingTower_EOL_primary_stainlessSteel = ((RecycledContentMetals * CoolingTower_EOL_stainlessSteel) - (RecyclingMetals2 * RecyclingEfficiencyMetals * CoolingTower_EOL_stainlessSteel))* SRmetals

CoolingTower_EOL_inc_PVC = CoolingTower_EOL_PVC * IncinerationPVC
CoolingTower_EOL_landfill_PVC = CoolingTower_EOL_PVC * LandfillPVC

CoolingTower_EOL_recycling_castiron = (RecyclingMetals2 * RecyclingEfficiencyMetals * CoolingTower_EOL_castiron) - (RecycledContentMetals * CoolingTower_EOL_castiron)
CoolingTower_EOL_primary_castiron = ((RecycledContentMetals * CoolingTower_EOL_castiron) - (RecyclingMetals2 * RecyclingEfficiencyMetals * CoolingTower_EOL_castiron)) * SRmetals

CoolingTower_EOL_recycling_airfun = (RecyclingMetals * RecyclingEfficiencyMetals * CoolingTower_EOL_airfun) - (RecycledContentMetals * CoolingTower_EOL_airfun)
CoolingTower_EOL_primary_airfun = ((RecycledContentMetals * CoolingTower_EOL_airfun) - (RecyclingMetals * RecyclingEfficiencyMetals * CoolingTower_EOL_airfun)) * SRmetals


## 3.10 Module D3 (related to the export of energy as a result of waste incineration) 

In [ ]:
CoolingTower_EOL_energyrecoveryheat_PVC = - CoolingTower_EOL_inc_PVC * LHVHDPE * Xincheat
CoolingTower_EOL_energyrecoveryele_PVC = - CoolingTower_EOL_inc_PVC * LHVHDPE * Xincele

# 4. Data for export 

In [ ]:
ThermalEnergyAbsorbed_CoolingTower = EnergyCooling * RSP